In [159]:
drive_path = '/Volumes/Brain2016'
# We need to import these modules to get started
import numpy as np
import pandas as pd
import os
import sys
from scipy.stats.stats import pearsonr
from scipy.stats import ttest_rel
# from scipy import signal
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
%matplotlib
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
# from allensdk.brain_observatory.drifting_gratings import DriftingGratings
import seaborn
import h5py

manifest_path = os.path.join(drive_path,'BrainObservatory','manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_path)
expMetaData = pd.DataFrame(boc.get_ophys_experiments())
cells = pd.DataFrame.from_records(boc.get_cell_specimens())
pearCor_DF = pd.DataFrame.from_csv('pearCor_DF.csv',sep=',')
runLen_DF = pd.read_csv('runHistograms/runLen_DF', delimiter='\t')
runLen_DF.drop('Unnamed: 0',1,inplace=True)

runCor_DF = pd.concat([pearCor_DF, runLen_DF], axis=1, join_axes=[runLen_DF.expID.index])
runCor_DF = pd.DataFrame.drop_duplicates(runCor_DF.T).T
runCor_DF.head()

for ind in runCor_DF.index:
    tmp = runCor_DF.iloc[ind].cell_pear_r
    runCor_DF.iloc[ind].cell_pear_r = [float(i.translate(None,'[]')) for i in tmp.split(',')]

Using matplotlib backend: MacOSX


In [247]:
np.sum(np.asarray(np.hstack(runCor_DF.cell_pear_r)) > .2)

286

(237,)

# Load Data

In [117]:
expContainerID = expMetaData.id[6]
# expContainerID = 501940850
exp = boc.get_ophys_experiment_data(ophys_experiment_id = expContainerID)
rSpd, _ = exp.get_running_speed()
_, dF = exp.get_dff_traces()

nanMask = ~np.isnan(rSpd)
rSpd = rSpd[nanMask]
rSpd = abs(rSpd)
dF = dF[:,nanMask]

/Users/bilalbari/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 115732 but corresponding boolean dimension is 115733


# Motion-Triggered Fluorescence

In [229]:
tmp = np.hstack(runCor_DF.cell_pear_r)
i = 5
while np.size(np.where(np.greater(runCor_DF.cell_pear_r[i],0.4))) == 0:
    i += 1
cell = np.where(np.greater(runCor_DF.cell_pear_r[i],0.4))[0][0]

expContainerID = runCor_DF.iloc[i].expID

exp = boc.get_ophys_experiment_data(ophys_experiment_id = expContainerID)
rSpd, t_rSpd = exp.get_running_speed()
t_dF, dF = exp.get_dff_traces()

nanMask = ~np.isnan(rSpd)
rSpd = rSpd[nanMask]
rSpd = abs(rSpd)
dF = dF[:,nanMask]

In [233]:
cell

149

## Motion Detection Algorithm

In [234]:
rThresh = 10
runStart = np.where(rSpd > rThresh); runStart = runStart[0];

firstInd = 0
secondInd = 1
runBouts = []
while secondInd < len(runStart):
    runEndFound = False
    if (runStart[secondInd] - runStart[firstInd]) <= 30: # run lengths < 1s long
        secondInd += 1
    elif (runStart[secondInd] - runStart[firstInd]) <= 60: # an appropriately long run bout
        iterInd1 = secondInd
        iterInd2 = iterInd1 + 1
        while ~runEndFound:
            if iterInd2 == len(runStart):
                runEndFound = True
                secondInd = iterInd2 + 1
                break
            if (runStart[iterInd2] - runStart[iterInd1]) <= 30:
                iterInd2 += 1
            elif (runStart[iterInd2] - runStart[iterInd1]) <= 60:
                iterInd1 = iterInd2 - 1
                iterInd2 += 1
            else:
                runEndFound = True
                runBouts.append(runStart[firstInd])
                firstInd = iterInd2
                secondInd = iterInd2 + 1
                break
    else:
        firstInd = secondInd 
        secondInd += 1

### Plot Motion Detection

In [235]:
fig,ax = plt.subplots()
print(len(runBouts))
ax.plot(rSpd,alpha = 0.7)
ax.scatter(runBouts,np.ones(len(runBouts))*rThresh,s=50,c='red')
ax.set_xlim(0, 50000)
ax.set_ylim(0, 50)

72


(0, 50)

In [236]:
locDF = pd.DataFrame(index=np.arange(len(runBouts)),columns=np.arange(dF.shape[0])) # runBouts, cellNum + 1 locomotion
for cell in range(dF.shape[0]):
    for locOnset in runBouts:
        locDF.loc[np.where(runBouts == locOnset)[0][0],cell] = dF[cell][locOnset-90:locOnset+90]
        
locDF['dx'] = ''
for locOnset in runBouts:
    locDF.loc[np.where(runBouts == locOnset)[0][0],'dx'] = rSpd[locOnset-90:locOnset+90]

In [237]:
fig,ax = plt.subplots(nrows = 3)
ax[0].plot(locDF.loc[:,'dx'].mean(),label='Running Speed (cm/s)')
ax[1].plot(locDF.loc[:,37].mean(),label='$\Delta$F/F$_0$')
ax[2].errorbar(x = 0, y = locDF.loc[:,37].mean()[:90].mean(), yerr = locDF.loc[:,37].mean()[:90].std()/np.sqrt(locDF.shape[0]),label="Mean Pre-Run $\Delta$F/F$_0$")
ax[2].errorbar(x = 1, y = locDF.loc[:,37].mean()[90:].mean(), yerr = locDF.loc[:,37].mean()[90:].std()/np.sqrt(locDF.shape[0]),label="Mean Post-Run $\Delta$F/F$_0$")
for currAx in ax:
    currAx.legend(loc='best',fontsize=12)
ax[2].set_xlim(-0.5, 1.5)

(-0.5, 1.5)

# Analyse All Data

In [333]:
fig,ax = plt.subplots(nrows = 3)

pearR_Thresh = 0.4
run_Thresh = 10

allPreRun = []
allPostRun = []

for expContainerID in expMetaData.id:
    print("expContainerID: %s" % (expContainerID))
    
    motionCells = np.where(np.greater(runCor_DF.cell_pear_r[ \
                runCor_DF.expID == expContainerID].values[0],pearR_Thresh))[0]
    if np.size(motionCells) != 0: # if any motion-sensitive cells here
        exp = boc.get_ophys_experiment_data(ophys_experiment_id = expContainerID)
        rSpd, _ = exp.get_running_speed()
        _, dF = exp.get_dff_traces()

        nanMask = ~np.isnan(rSpd)
        rSpd = rSpd[nanMask]
        dF = dF[:,nanMask]


        runStart = np.where(rSpd > run_Thresh); runStart = runStart[0];

        firstInd = 0
        secondInd = 1
        runBouts = []
        while secondInd < len(runStart):
            runEndFound = False
            if (runStart[secondInd] - runStart[firstInd]) <= 30: # run lengths < 1s long
                secondInd += 1
            elif (runStart[secondInd] - runStart[firstInd]) <= 60: # an appropriately long run bout
                iterInd1 = secondInd
                iterInd2 = iterInd1 + 1
                while ~runEndFound:
                    if iterInd2 == len(runStart):
                        runEndFound = True
                        secondInd = iterInd2 + 1
                        break
                    if (runStart[iterInd2] - runStart[iterInd1]) <= 30:
                        iterInd2 += 1
                    elif (runStart[iterInd2] - runStart[iterInd1]) <= 60:
                        iterInd1 = iterInd2 - 1
                        iterInd2 += 1
                    else:
                        runEndFound = True
                        runBouts.append(runStart[firstInd])
                        firstInd = iterInd2
                        secondInd = iterInd2 + 1
                        break
            else:
                firstInd = secondInd 
                secondInd += 1

        locDF = pd.DataFrame(index=np.arange(len(runBouts)),columns=np.arange(len(motionCells))) # runBouts, cellNum + 1 locomotion
        for cell in range(len(motionCells)):
            for locOnset in runBouts:
                locDF.loc[np.where(runBouts == locOnset)[0][0],cell] = dF[cell][locOnset-90:locOnset+90]

        locDF['dx'] = ''
        for locOnset in runBouts:
            locDF.loc[np.where(runBouts == locOnset)[0][0],'dx'] = rSpd[locOnset-90:locOnset+90]
        
        if locDF.loc[0].values[0].size == 0:
            locDF = locDF.drop([0]);
            
        for cell in range(locDF.shape[1]-1):
#             ax[0].plot(locDF.loc[:,'dx'].mean(),label='Running Speed (cm/s)')
#             ax[1].plot(locDF.loc[:,cell].mean(),label='$\Delta$F/F$_0$')
#             ax[2].errorbar(x = 0, y = locDF.loc[:,cell].mean()[:90].mean(), yerr = locDF.loc[:,cell].mean()[:90].std()/np.sqrt(locDF.shape[0]),label="Mean Pre-Run $\Delta$F/F$_0$")
#             ax[2].errorbar(x = 1, y = locDF.loc[:,cell].mean()[90:].mean(), yerr = locDF.loc[:,cell].mean()[90:].std()/np.sqrt(locDF.shape[0]),label="Mean Post-Run $\Delta$F/F$_0$")
#             for currAx in ax:
#                 currAx.legend(loc='best',fontsize=12)
#             ax[2].set_xlim(-0.5, 1.5)
#             plt.pause(1.0)
#             for currAx in ax:
#                 currAx.cla()
                
            allPreRun.append(locDF.loc[:,cell].mean()[:75].mean())
            allPostRun.append(locDF.loc[:,cell].mean()[105:].mean())
#             print("Cell Added")

expContainerID: 502066273
expContainerID: 506278598
expContainerID: 510529677
expContainerID: 504614224
expContainerID: 501724959
expContainerID: 506156402
expContainerID: 508366303
expContainerID: 510093797
expContainerID: 507990552
expContainerID: 508596945
expContainerID: 507464107
expContainerID: 501929146
expContainerID: 510536157
expContainerID: 507881529
expContainerID: 509729072
expContainerID: 507945410
expContainerID: 510221121
expContainerID: 503019786
expContainerID: 508753256
expContainerID: 509799475
expContainerID: 501254258
expContainerID: 496934409
expContainerID: 506773892
expContainerID: 501794235
expContainerID: 505696248
expContainerID: 504101079
expContainerID: 501940850
expContainerID: 504508104
expContainerID: 507691036
expContainerID: 502741583
expContainerID: 505695962
expContainerID: 501729039
expContainerID: 501498760
expContainerID: 496935917
expContainerID: 505198966
expContainerID: 512149367
expContainerID: 501474098
expContainerID: 501021421
expContainer

/Users/bilalbari/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 105734 but corresponding boolean dimension is 105735


expContainerID: 511856569
expContainerID: 503820068
expContainerID: 502115959
expContainerID: 509295073
expContainerID: 510518066
expContainerID: 508408956
expContainerID: 497060401
expContainerID: 504115289
expContainerID: 508378520
expContainerID: 506074436
expContainerID: 502667200
expContainerID: 509737584
expContainerID: 510517609


/Users/bilalbari/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 113862 but corresponding boolean dimension is 113863


expContainerID: 510705057
expContainerID: 511573879
expContainerID: 501490610
expContainerID: 497256116
expContainerID: 505407318
expContainerID: 501933264
expContainerID: 500947031
expContainerID: 510917254
expContainerID: 505693621
expContainerID: 506954308
expContainerID: 500860585
expContainerID: 501567237
expContainerID: 501717543
expContainerID: 501484643
expContainerID: 511440894
expContainerID: 506356888
expContainerID: 506539927
expContainerID: 510938357
expContainerID: 512176430
expContainerID: 501886692
expContainerID: 510524416
expContainerID: 510699005
expContainerID: 506248008
expContainerID: 511434920
expContainerID: 502810282
expContainerID: 506030579
expContainerID: 501132496
expContainerID: 509881188
expContainerID: 504642019
expContainerID: 511305590
expContainerID: 506809539
expContainerID: 510532780
expContainerID: 502526200
expContainerID: 512311673
expContainerID: 504639393
expContainerID: 501567645
expContainerID: 502205092
expContainerID: 502382906
expContainer

/Users/bilalbari/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 105720 but corresponding boolean dimension is 105721


expContainerID: 508262069
expContainerID: 510234687
expContainerID: 507552264
expContainerID: 504646202
expContainerID: 510021399
expContainerID: 509336418
expContainerID: 502352946
expContainerID: 506441755
expContainerID: 510514770
expContainerID: 501574836
expContainerID: 510514430
expContainerID: 509600709
expContainerID: 511596055


/Users/bilalbari/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 105960 but corresponding boolean dimension is 105961


expContainerID: 503864409
expContainerID: 504914156
expContainerID: 501788003
expContainerID: 509339069
expContainerID: 502254330
expContainerID: 510706209
expContainerID: 505845219
expContainerID: 510214538
expContainerID: 507304910
expContainerID: 512124564
expContainerID: 509630631
expContainerID: 504108263
expContainerID: 510514474
expContainerID: 505687912
expContainerID: 500855614
expContainerID: 504593468
expContainerID: 508615592
expContainerID: 501839084
expContainerID: 509841198
expContainerID: 512164988
expContainerID: 506540916
expContainerID: 505811062
expContainerID: 506520703
expContainerID: 509522931
expContainerID: 508396265
expContainerID: 504637623
expContainerID: 509523145
expContainerID: 511595995
expContainerID: 501271265
expContainerID: 502383036
expContainerID: 503538804
expContainerID: 504625475
expContainerID: 509962140
expContainerID: 504568756
expContainerID: 508563988
expContainerID: 504809131
expContainerID: 501773889
expContainerID: 510814438
expContainer

/Users/bilalbari/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 105738 but corresponding boolean dimension is 105739


In [334]:
print(np.asarray(allPreRun).mean())
print(np.asarray(allPostRun).mean())

0.0090507
0.00777999
